## Outperform Chalk Notebook

This notebook carries out the asynchronous algorithm described in the Making Sense of the Madness: A Bilevel Programming Approach to Bracket Generation paper. 

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import time
import random
import pickle

# MILP Optimization API
from pyscipopt import Model

# Genetic algorithm package
import leap_ec
from leap_ec.decoder import IdentityDecoder
from leap_ec.representation import Representation
from leap_ec import ops
from leap_ec.real_rep.ops import mutate_gaussian
from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.problem import FunctionProblem
from leap_ec.distrib import DistributedIndividual, asynchronous
from leap_ec.distrib.probe import log_worker_location

# Asynchronous or distributed workers
from dask.distributed import Client, LocalCluster

#Import helper functions
from opt_functions import (
    opt_bracket,
    get_results,
    brack_score,
    sigmoid
)

### Import relevant data

We'll need to import the data that will be used during the optimization. This includes:
* `data_frames`: This dictionary contains a dataframe of important data for each year. The information contained in these dataframes for each year are:
    * Probabilistic projections for each team in each round
    * Matrix representing the actual outcome of the tournament
    * Matrix representing the chalk outcome of the tournament
* `chalk_total_scores`: This dictionary contains the scores for the chalk brackets to be used during fitness function evaluation.
* `best_d`: This is the best distance vector from a previous run. Provides a starting point for the optimization. This is not required; a rendomly initialized distance vector can be used.

In [2]:
# Import relevant data for optimization

years = ["2014", "2015", "2016", "2017", "2018", "2019", "2021", "2022", "2023"]  #Years to be analyzied

# Import data frames with probabilities, chalk, and actual outcomes
cwd = os.getcwd()
file_path = "\Data\\final_bracket_data\\"
data_frames = {}
for year in years:
    data_frames[year] = pd.read_csv(cwd + file_path + year + ".csv")

#Record the scores for the chalk brackets
chalk_total_scores = {"2014": 690,
                "2015": 860,
                "2016": 990,
                "2017": 700,
                "2018": 1140,
                "2019": 950,
                "2021": 870,
                "2022": 790,
                "2023": 520}

# Extract the most recent best distances
with open('best_d_final.pkl', 'rb') as f:
    best_d = pickle.load(f)
f.close()

#Set up bounds for individual initialization using best distance vector as baseline
init_bounds = [(d - 0.001, d + 0.001) for d in best_d ]
print(best_d)

[0.9992638406776697, 0.9093473957915438, 0.928538915959301, 0.9153074542915343, 0.8261114325054147, 0.9826968464863475]


### Define fitness function

We need to define the fitness function that will be used for evaluation in the genetic algorithm. THere are several options in the cell below:
* `fitness_avg()`: This calculates the average score of the brackets generated each year. More formally this is: $\frac{1}{K}\sum_{k=1}^{K} S(\mathbf{X}_{k}^{*})$. This fitness function completely ignores the idea of outperforming chalk
* `fitness_num_better()`: This fitness function evaluates the number of times that a generated bracket outperforms chalk. More formally $\sum_{k=1}^{K} g\left(\mathbf{X}_k^*\right) = 
\begin{cases} 
1, & \text{if } S\left(\mathbf{X}_k^*\right) \geq S\left(\mathbf{C}_k\right) \\
0, & \text{otherwise}
\end{cases}$. This is what we're looking for but given the small dataset, it is too discrete and makes it challenging to optimize.
* `fitness_avg_diff()`: This fitness function attempts to add more topology to the search space. It calculates the average outperformance in points over the years. More formally: $\frac{1}{K}\sum_{k=1}^{K} S(\mathbf{X}_{k}^{*}) - S(\mathbf{C}_{k}^{*})$. This method provides more topology but does not accomplish the true goal of maximizing the number of times we outperform chalk.
* `fitness_sigmoid`(): This is the final selected fitness function. This fitness function provides the topology needed while also putting an emphasis on outperforming chalk by passing the difference through a sigmoid function. More formally $$\sum_{k=1}^{K} \frac{1}{1 + e^{-\frac{\left(S\left(\mathbf{X}_k^*\right) - S\left(\mathbf{C}_k\right)\right)}{w}}}$$. 

In [8]:
# Genetic algorithm fitness functions
def fitness_avg(phenome: list[float]) -> float:
    """
    Calculate fitness using average score

    Parameters
    ----------
    phenome : list[float]
        distance vector

    Returns
    -------
    float
        fitness score
    """
    fitness = 0
    for year in data_frames.keys():
        output, x = opt_bracket(data_frames[year], phenome)
        results = get_results(output, x)
        fitness += sum(brack_score(results, data_frames[year]))
    return fitness/len(data_frames)

def fitness_num_better(phenome: list[float]) -> int:
    """
    Calculate fitness using number of times outperforming chalk

    Parameters
    ----------
    phenome : list[float]
        distance vector

    Returns
    -------
    int
        fitness score
    """
    fitness = 0
    for year in data_frames.keys():
        output, x = opt_bracket(data_frames[year], phenome)
        results = get_results(output, x)
        if sum(brack_score(results, data_frames[year])) > chalk_total_scores[year]:
            fitness+=1
    return fitness

def fitness_avg_diff(phenome: list[float]) -> float:
    """
    Calculate fitness using average score aboove chalk

    Parameters
    ----------
    phenome : list[float]
        distance vector

    Returns
    -------
    float
        fitness score
    """
    fitness = 0
    for year in data_frames.keys():
        output, x = opt_bracket(data_frames[year], phenome)
        results = get_results(output, x)
        fitness += sum(brack_score(results, data_frames[year])) - chalk_total_scores[year]
    return fitness/len(data_frames)

def fitness_sigmoid(phenome: list[float]) -> float:
    """
    Calculate fitness using sigmoid difference above chalk

    Parameters
    ----------
    phenome : list[float]
        distance vector

    Returns
    -------
    float
        fitness score
    """
    fitness = 0
    for year in data_frames.keys():
        output, x = opt_bracket(data_frames[year], phenome)
        results = get_results(output, x)
        diff = sum(brack_score(results, data_frames[year])) - chalk_total_scores[year]
        fitness += sigmoid(diff, 25)
    return fitness

### Asynchronous genetic algorithm

We now carry out the asynchronous genetic algorithm. The algorithm has the following specification:
* Max births: 2048
* Population size: 16 individuals
* Fitness function: Sigmoid
* Parent selection: Tournament selection
* Crossover: Uniform crossover probability across all elements
* Mutation: Addition of a gaussian random variable two around two elements

Results of the run are printed out to csv file named `test.csv`

In [9]:
#Asynchronous distributed genetic algorithm

MAX_BIRTHS = 2048
INIT_POP_SIZE = 16
POP_SIZE = 16
file = open("test.csv", "w")

tic = time.perf_counter()
with Client(LocalCluster()) as client:
    final_pop = asynchronous.steady_state(client, # dask client
                                  max_births=MAX_BIRTHS,
                                  init_pop_size=INIT_POP_SIZE,
                                  pop_size=POP_SIZE,

                                  representation=Representation(
                                  decoder=IdentityDecoder(),             # Genotype and phenotype are the same for this task
                                  #initialize=create_real_vector(bounds=[(0.8, 1)]*6),  # Initial genomes are random gaussian
                                  initialize=create_real_vector(bounds=init_bounds),
                                  individual_cls=DistributedIndividual),

                                  problem=FunctionProblem(fitness_sigmoid, maximize=True),

                                  offspring_pipeline=[ops.tournament_selection, # Select parents via tournament_selection selection
                                  ops.clone,                                    # Copy them (just to be safe)
                                  ops.uniform_crossover,                         # Crossover each element with probability 0.2
                                  mutate_gaussian(std=.1/3, expected_num_mutations=2, hard_bounds=(0.8, 1)),                 # Basic mutation: gaussian mutation        
                                  ops.pool(size=1)             # Collect offspring into a new population
                                  ],
                                    
                                  evaluated_probe=log_worker_location(file))

toc = time.perf_counter()
file.close()
client.shutdown()
print("Solve time was:{} seconds".format(toc-tic))

Solve time was:3546.7594076000387 seconds


### Results

We now retrieve the best individual from the final population and print the distance vector and its fitnessz

In [6]:
# Retrieve the best individual
max_ind = [ind.fitness for ind in final_pop].index(max([ind.fitness for ind in final_pop]))
print(max_ind)
ga_best_d = final_pop[max_ind].genome
print(ga_best_d)
print(final_pop[max_ind].fitness)
print(fitness_num_better(final_pop[max_ind].genome))

0
[0.99926384 0.9093474  0.92853892 0.91530745 0.82611143 0.98269685]
6.484679406331287
7


### Comparison to chalk

Now let's compare the brackets generated by this distance vector to chalk

In [24]:
# See performance
d = best_d
round_scores = []
chalk_round_scores = []
for year in years:
    output, x = opt_bracket(data_frames[year], d)
    results = get_results(output, x)
    print(year + ":")
    d_scores = brack_score(results, data_frames[year])
    chalk_scores = brack_score(data_frames[year].iloc[:, 14:20], data_frames[year])
    print(f"d score is: {np.round(sum(d_scores))}")
    print(f"chalk score is: {chalk_total_scores[year]}")
    print('\n')
    round_scores.append(d_scores)
    chalk_round_scores.append(chalk_scores)

2014:
d score is: 780.0
chalk score is: 690


2015:
d score is: 870.0
chalk score is: 860


2016:
d score is: 1050.0
chalk score is: 990


2017:
d score is: 860.0
chalk score is: 700


2018:
d score is: 710.0
chalk score is: 1140


2019:
d score is: 1250.0
chalk score is: 950


2021:
d score is: 1020.0
chalk score is: 870


2022:
d score is: 1020.0
chalk score is: 790


2023:
d score is: 510.0
chalk score is: 520




In [27]:
np.round(round_scores) - np.round(chalk_round_scores)

array([[  10.,   40.,   40.,    0.,    0.,    0.],
       [  10.,   40.,  -40.,    0.,    0.,    0.],
       [   0.,   20.,  -40.,   80.,    0.,    0.],
       [   0.,    0.,    0.,    0.,  160.,    0.],
       [  10.,   40.,    0.,    0., -160., -320.],
       [  20.,  -80.,  -40.,   80.,    0.,  320.],
       [  10.,   20.,   40.,   80.,    0.,    0.],
       [ -10.,  -40.,  -40.,    0.,    0.,  320.],
       [ -10.,  -40.,   40.,    0.,    0.,    0.]])